In [1]:
# !pip install tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# Only if you have a GPU

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
######## GPU CONFIGS FOR RTX 2070 ###############
## Please ignore if not training on GPU       ##
## this is important for running CuDNN on GPU ##

# check if GPU can be seen by TF
tf.config.list_physical_devices('GPU')
# enable this line if you wish to see whether GPU/CPU executes
# each command
#tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
###############################################

1 Physical GPUs, 1 Logical GPU


# Load Data

In [4]:
# see available tfds data sets
", ".join(tfds.list_builders())

'abstract_reasoning, aeslc, aflw2k3d, ai2_arc, amazon_us_reviews, anli, arc, bair_robot_pushing_small, beans, big_patent, bigearthnet, billsum, binarized_mnist, binary_alpha_digits, blimp, c4, caltech101, caltech_birds2010, caltech_birds2011, cars196, cassava, cats_vs_dogs, celeb_a, celeb_a_hq, cfq, chexpert, cifar10, cifar100, cifar10_1, cifar10_corrupted, citrus_leaves, cityscapes, civil_comments, clevr, clinc_oos, cmaterdb, cnn_dailymail, coco, coil100, colorectal_histology, colorectal_histology_large, common_voice, cos_e, cosmos_qa, covid19sum, crema_d, curated_breast_imaging_ddsm, cycle_gan, deep_weeds, definite_pronoun_resolution, dementiabank, diabetic_retinopathy_detection, div2k, dmlab, downsampled_imagenet, dsprites, dtd, duke_ultrasound, emnist, eraser_multi_rc, esnli, eurosat, fashion_mnist, flic, flores, food101, forest_fires, fuss, gap, geirhos_conflict_stimuli, german_credit_numeric, gigaword, glue, groove, higgs, horses_or_humans, i_naturalist2017, imagenet2012, imagene

In [5]:
# using TFDS dataset
# note: as_supervised converts dicts to tuples
imdb_train, ds_info = tfds.load(name="imdb_reviews", split="train", 
                                with_info=True, as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews", split="test", 
                      as_supervised=True)

In [6]:
print(ds_info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [7]:
for example, label in imdb_train.take(1):
    print(example, '\n', label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


In [8]:
# Use the default tokenizer settings
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
MAX_TOKENS = 0

for example, label in imdb_train:
  some_tokens = tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_tokens):
        MAX_TOKENS = len(some_tokens)
  vocabulary_set.update(some_tokens)

In [9]:
imdb_encoder = tfds.features.text.TokenTextEncoder(vocabulary_set, 
                                                   tokenizer=tokenizer)
vocab_size = imdb_encoder.vocab_size

print(vocab_size, MAX_TOKENS)

93931 2525


In [10]:
# Lets verify tokenization and encoding works
for example, label in imdb_train.take(1):
    print(example)
    encoded = imdb_encoder.encode(example.numpy())
    print(imdb_encoder.decode(encoded))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
This was an absolutely terrible movie Don t be lured in by Christopher Walken or Michael Ironside Both are great actors but this must simply be their worst role in history Even their great acting could not redeem this movie s ridiculous storyline This 

In [11]:
# transformation fucntions to be used with the dataset
from tensorflow.keras.preprocessing import sequence

def encode_pad_transform(sample):
    encoded = imdb_encoder.encode(sample.numpy())
    pad = sequence.pad_sequences([encoded], padding='post', 
                                 maxlen=150)
    return np.array(pad[0], dtype=np.int64)  


def encode_tf_fn(sample, label):
    encoded = tf.py_function(encode_pad_transform, 
                                       inp=[sample], 
                                       Tout=(tf.int64))
    encoded.set_shape([None])
    label.set_shape([])
    return encoded, label

In [12]:
# test the transformation on a small subset
subset = imdb_train.take(10)
tst = subset.map(encode_tf_fn)

In [13]:
for review, label in tst.take(1):
    print(review, label)
    print(imdb_encoder.decode(review))

tf.Tensor(
[11656 31784 37897 22934 71575 81905 19681 30209 57969 20389 12638 22613
 21862  5499 22944 58608  2155 71664 83985 23849 80304 23571 48259 81535
 49548 57969 37115 78725  6916 12638 43953 58008 37115 23849 92552 69141
 45856 53359 48259 81905 56690 43180 51447 11656 81905 50648 37897 84637
 80835 45507 14443 27814 35500 29509 36671 68930 91084 91855 73442 20914
 71778 42677 91084 45537 37115 78395 13018 27293 88812 38487 32134 49986
 33801 31800 24165 57465 89656 38318 87284  5499 31784 28425 23571 21457
 36671 45794 69918 12638 21457 81905 31587 31784 36823 14883 49480 21767
 32097 90892 89245 52162 31587 90597 83985 23628 46411 48259  8865  8715
 56690 46411 21862  5499 56690 23123 18741 90892 69141 18287 49825 39206
 46121     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
This was an

In [14]:
#now tokenize/encode/pad all training
# and testing data
encoded_train = imdb_train.map(encode_tf_fn,
                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test = imdb_test.map(encode_tf_fn,
                             num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Preparing the model

In [15]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=100

In [16]:
def build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model


In [17]:
model = build_model_lstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (100, None, 64)           6011584   
_________________________________________________________________
lstm (LSTM)                  (100, 64)                 33024     
_________________________________________________________________
dense (Dense)                (100, 1)                  65        
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

# Model Training and Evaluation

In [19]:
# Prefetch for performance
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [20]:
model.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 21s 84ms/step - loss: 0.4106 - accuracy: 0.8049 - precision: 0.7849 - recall: 0.8400
Epoch 2/10
250/250 [==============================] - 22s 88ms/step - loss: 0.1692 - accuracy: 0.9380 - precision: 0.9403 - recall: 0.9353
Epoch 3/10
250/250 [==============================] - 22s 88ms/step - loss: 0.1109 - accuracy: 0.9622 - precision: 0.9640 - recall: 0.9603
Epoch 4/10
250/250 [==============================] - 22s 88ms/step - loss: 0.0674 - accuracy: 0.9777 - precision: 0.9781 - recall: 0.9773
Epoch 5/10
250/250 [==============================] - 22s 87ms/step - loss: 0.0552 - accuracy: 0.9823 - precision: 0.9819 - recall: 0.9828
Epoch 6/10
250/250 [==============================] - 23s 90ms/step - loss: 0.0654 - accuracy: 0.9774 - precision: 0.9770 - recall: 0.9779
Epoch 7/10
250/250 [==============================] - 22s 89ms/step - loss: 0.0257 - accuracy: 0.9924 - precision: 0.9921 - recall: 0.9928
Epoch 8/10
250/250 [=======

In [21]:
model.evaluate(encoded_test.batch(BATCH_SIZE))

250/250 [==============================] - 21s 82ms/step - loss: 0.9765 - accuracy: 0.8317 - precision: 0.8013 - recall: 0.8822


[0.9764885902404785,
 0.8317199945449829,
 0.8013225793838501,
 0.8821600079536438]

# BiLSTM Model

In [22]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=50

In [23]:
dropout=0.2
def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, return_sequences=True, dropout=0.25)),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, dropout=0.25)),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model

In [24]:
bilstm = build_model_bilstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

bilstm.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (50, None, 128)           12023168  
_________________________________________________________________
dropout (Dropout)            (50, None, 128)           0         
_________________________________________________________________
bidirectional (Bidirectional (50, None, 128)           98816     
_________________________________________________________________
dropout_1 (Dropout)          (50, None, 128)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (50, 128)                 98816     
_________________________________________________________________
dropout_2 (Dropout)          (50, 128)                 0         
_________________________________________________________________
dense_1 (Dense)              (50, 1)                  

In [25]:
bilstm.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

In [26]:
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [27]:
bilstm.fit(encoded_train_batched, epochs=5)

Epoch 1/5
500/500 [==============================] - 68s 136ms/step - loss: 0.3763 - accuracy: 0.8266 - precision: 0.8205 - recall: 0.8362
Epoch 2/5
500/500 [==============================] - 69s 138ms/step - loss: 0.1505 - accuracy: 0.9465 - precision: 0.9463 - recall: 0.9467
Epoch 3/5
500/500 [==============================] - 69s 137ms/step - loss: 0.0701 - accuracy: 0.9761 - precision: 0.9765 - recall: 0.9757
Epoch 4/5
500/500 [==============================] - 68s 137ms/step - loss: 0.0724 - accuracy: 0.9753 - precision: 0.9747 - recall: 0.9758
Epoch 5/5
500/500 [==============================] - 69s 137ms/step - loss: 0.0416 - accuracy: 0.9859 - precision: 0.9856 - recall: 0.9862


In [28]:
bilstm.evaluate(encoded_test.batch(BATCH_SIZE))

500/500 [==============================] - 17s 35ms/step - loss: 0.6753 - accuracy: 0.8384 - precision: 0.8459 - recall: 0.8274


[0.6753188967704773,
 0.8383600115776062,
 0.8459147810935974,
 0.8274400234222412]